In [6]:
import random
from deap import base
from deap import creator
from deap import tools
from deap import gp
import array
import numpy
import operator
from deap import gp
import json

# #-----------  GP faiblement typé ----------- # #

In [9]:
pset = gp.PrimitiveSet("main", 2) # crée un ensemble de primitive
#ses arguments sont le nom de la procédure qu'elle va générer "main"
#et son nombre d'entrée 2

#ajoute des fonctions comme primitives 
# le premier argument est la fonction à affihcer "max"
# le second est le nombre d'input/argument
pset.addPrimitive(max, 2) 
pset.addPrimitive(operator.add, 2)  
pset.addPrimitive(operator.mul, 2)

#ajoute une constante terminale
pset.addTerminal(3)

In [10]:
pset.renameArguments(ARG0="x")
pset.renameArguments(ARG1="y")
#Dans ce cas, toutes les fonctions prennent deux arguments. 

In [13]:
#Une fonction de négation à 1 argument, par exemple, 
#pourrait être réalisée avec :
#pset.addPrimitive(operator.neg, 1)

In [15]:
expr = gp.genFull(pset, min_=1, max_=3) 
tree = gp.PrimitiveTree(expr)

# #-----------  GP Fortement typé ----------- # #


In [17]:
def if_then_else(input, output1, output2):
    # nous définissons d'abord une fonction if then else qui renvoie 
    # le deuxième argument si le premier est vrai 
    # et le troisième sinon.
    return output1 if input else output2


# Ensuite, nous définissons notre PrimitiveSetTyped. 
# Encore une fois, la procédure est nommée "main".
# Le deuxième argument définit les types d'entrée du programme. 
# Ici, "x" est un bool et "y" est un float. 
# Le troisième argument définit le type de sortie du programme comme étant un float. 
pset = gp.PrimitiveSetTyped("main", [bool, float], float)

# Pour ajouter des primitives à cette primitive, il faut maintenant définir les types d'entrée et 
# de sortie des primitives et du terminal. 
pset.addPrimitive(operator.xor, [bool, bool], bool)
pset.addPrimitive(operator.mul, [float, float], float)

# Par exemple, nous définissons le premier argument de notre fonction "if_then_else" 
# comme un booléen, les deuxième et troisième arguments doivent être des flottants. 
pset.addPrimitive(if_then_else, [bool, float, float], float)
pset.addTerminal(3.0, float)
pset.addTerminal(1, bool)
pset.renameArguments(ARG0="x")
pset.renameArguments(ARG1="y")

# La fonction est définie comme renvoyant un flottant. 
# Nous comprenons maintenant que la primitive de multiplication ne peut avoir comme entrée que le
# terminal 3.0, la fonction if_then_else ou le "y", qui sont les seuls flottants définis.


# #-----------  Constante éphémère ----------- # #

In [23]:
pset.addEphemeralConstant("EphémèreTest",lambda: random.randint(-10, 10), int)

# #-----------  Generation of Tree Individuals ----------- # #

In [24]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin,
               pset=pset)

In [26]:
# Nous enregistrons ensuite les fonctions de génération dans une Toolbox.
toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual,
                 toolbox.expr)
# L'appel de toolbox.individual() retourne facilement un individu de type PrimitiveTree.

# #-----------  Evaluation of Trees ----------- # #

In [33]:
# Par exemple, les lignes suivantes génèrent un arbre et produisent le code du 
# premier exemple d'ensemble de primitives.
expr = gp.genFull(pset, min_=1, max_=3)
tree = gp.PrimitiveTree(expr)
str(tree)

'mul(mul(3.0, y), mul(y, y))'

In [ ]:
# rappel attribut mode de compile :
#      mode : Soit exec ou eval ou simple.
#             eval - n'accepte qu'une seule expression.
#             exec - Il peut prendre un bloc de code qui contient des instructions, des classes et des fonctions Python, etc.
#             single - s'il consiste en une seule instruction interactiv

In [41]:
function = compile(tree, pset)   
                             #La fonction prend troie arguments : 
                                   # l'expression à compiler
                                   # l'ensemble de primitives associé. 
                                   # le mode choisi  
function(1, 2)

TypeError: compile() missing required argument 'mode' (pos 3)